In [ ]:
### colab 사용 시 실행
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.chdir('gdrive/My Drive/Google Colaboratory/kakao_arena/kakao_arena_melon/src/ipynb')

In [ ]:
import sys
sys.path.append('../')
from basic_utils import *

from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as spr
import pickle, psutil, random

import keras 
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout

In [ ]:
data_path = '../../../1_melon_playlist/data/'   # data 경로

train = pd.read_json(data_path+'train.json')
val = pd.read_json(data_path+'val.json')
# test = pd.read_json(data_path+'test.json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
# tag_song_dict = load_json(data_path+'tag_song_dict_no0.json')
# song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
no_song_idx, no_tag_idx, no_both_idx, yes_all_idx = check_target_type(val)

노래가 하나도 없음 :  2630
노래, 태그 둘 다 존재함 :  8975
태그가 없음 :  9661
둘다 없음 :  1749


In [ ]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))

year_list = list(train['year'].unique())
year_list= sorted(year_list)

small_years = [19]

In [ ]:
## TRAIN
print("train only year", train[(train['year'].isin(small_years))].shape)
train_tmp = train[(train['year'].isin(small_years))]

# tmp = train.copy()
# like_num = int(np.percentile(train_tmp['like_cnt'], 10))
# print("like_num_cutpoint : ",like_num)
train_tmp = train_tmp[(train_tmp['like_cnt'] > 0)]
train_tmp.reset_index(inplace = True, drop = True)
print("train best playlist", train_tmp.shape)

# train_tmp = train.copy()
# like_num = int(np.percentile(train_tmp['like_cnt'], 10))
# print("like_num_cutpoint : ",like_num)
# train_tmp.reset_index(inplace = True, drop = True)
# print("train best playlist", train_tmp.shape)

train only year (21289, 7)
train best playlist (17850, 7)


In [ ]:
c = Counter()

for doc in train_tmp['songs']:
    c.update(doc)

tmp_dict= pd.DataFrame.from_dict(c, orient = 'index')
print("before ", tmp_dict.shape)
tmp_dict = tmp_dict[tmp_dict[0] > 5]
print("after ", tmp_dict.shape)
tmp_dict.reset_index(inplace = True)

## 5회 이상 등장한 노래만 사용! ==> use_song에 list형태로 담음
use_song = list(tmp_dict['index'])

before  (225128, 1)
after  (30355, 1)


In [ ]:
def before_pivot(data):
    result = []
    start = time.time()  # 시작 시간 저장     
    
    for i in tqdm(range(len(data))):
        for song in data.loc[i:i, 'songs'].values[0]:
            if song in use_song:
                df = pd.DataFrame({
                            'id' : [data.loc[i:i, 'id'].values[0]],
                            'song' : [song],
                            'point' : 1
                        })
                result += [df]

    final_result = pd.concat(result)
    print(final_result.shape)
    end = time.time()  # 시작 시간 저장     
    print("spend time ", round((end-start)/60, 2))
    return final_result

In [ ]:
train_df = before_pivot(train_tmp)
print('complete train')
# val_df = before_pivot(val_tmp)

del train_tmp 
gc.collect()

100%|██████████| 17850/17850 [15:15<00:00, 19.50it/s]


(616300, 3)
spend time  16.17
complete train


0

In [ ]:
# all_df = pd.concat([train_df, val_df])
all_df = train_df.copy()
all_df = all_df.pivot(index = 'song', columns ='id', values = 'point').fillna(0)
all_df.columns = all_df.columns.astype(str)

In [ ]:
all_df.head(2)

id,14,40,41,42,43,55,62,67,112,123,...,153345,153350,153355,153364,153368,153378,153379,153390,153415,153423
song,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# train_DF = all_df[all_df.index.isin(train_df['id'].to_list())]
# val_DF = all_df[all_df.index.isin(val_df['id'].to_list())]

In [ ]:
# display(train_DF.head(2))
# display(val_DF.head(2))

In [ ]:
# ## 5회 이상 존재하지 않는 노래는 삭제 시킴 => 일부 plyst는 아예 소멸됨(비주류의 노래만 갖고 있어서) => 복원 시켜야 함 
# print(set(val_tmp['id'].to_list()) - set(val_DF.index.to_list()))

In [ ]:
# loose_val_data = pd.DataFrame(list(set(val_tmp['id'].to_list()) - set(val_DF.index.to_list())))
# loose_val_data.columns = ['id']

In [ ]:
all_df.to_parquet(data_path+'train_DF.parquet')
# val_DF.to_parquet(data_path+'val_DF.parquet')
# loose_val_data.to_csv(data_path+'loose_val_data.csv')